In [1]:
# - - - - - - - - - - - - - - - - Install dropbox - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

pip install dropbox

Note: you may need to restart the kernel to use updated packages.


In [16]:
# - - - - - - - - - - - - - - - - Import libraries - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

import dropbox
import pandas as pd
import io

from pathlib import Path

import math
import numpy as np


In [17]:
# - - - - - - - - - - - - - - - - Read files from DropBox and combine into one file per data type - - - - - - - -

# Important:
# 1. Save a copy of this script to your own Google Drive
# 2. Genereate your own DROPBOX_TOKEN,
#    NEVER share it with others or check into version control systems like GitHub

dbx = dropbox.Dropbox('') ## REPLACE TOKEN HERE
dbx.users_get_current_account()


# initialize dataframes
data_blank = [[]]
dfp_main = pd.DataFrame(data_blank)
dfp_logs = pd.DataFrame(data_blank)
dfp_grades = pd.DataFrame(data_blank)


# define function to parse term code from file path string
def set_term_code(file_path,year) :
    term_code = ''
    if 'fall' in file_path :
        term_code = year + '10'
        term_code = int(term_code) + 100
        term_code = str(term_code)
    elif 'winter' in file_path :
        term_code = year + '20'
    else :
        print('Error reading term description from file path')
    return term_code


# define function to translate term code to term description
def get_term_desc(term_code) :
    term_type = int(term_code)%100
    year = math.trunc(int(term_code)/100)
    if term_type == 10 :
        year -= 1
        term_typ_desc = 'Fall'
    else :
        term_typ_desc = 'Winter'
    term_desc = term_typ_desc + ' ' + str(year)
    return(term_desc)


# read from folder in dropbox
for entry in dbx.files_list_folder('/MyLA Data Identified Subset').entries:
    file_path = entry.path_lower
    
    
    # check for course id map file name to save dataframe for later joining
    if 'course_map' in file_path and 'winter2019' in file_path :
        _, res = dbx.files_download(file_path)
        with io.BytesIO(res.content) as stream:
            df = pd.read_csv(stream)
        df_cmap = df
        print(' - - - course map file')
        df_cmap = df_cmap.rename({'Canvas Course ID': 'Canvas CID','Canvas Course Name':'Canvas CName'}
                                 , axis='columns')
        df_cmap['Canvas CID'] = df_cmap['Canvas CID'].astype(str)
        
        
    # check for student id map file name to save dataframe for later joining
    if 'student_map' in file_path :
        _, res = dbx.files_download(file_path)
        with io.BytesIO(res.content) as stream:
            df = pd.read_csv(stream)
        df_smap = df
        print(' - - - student map file')
        df_smap.columns = ['Uniqname','Student SID']
        

    # check for student grades data and compile into one frame
    elif 'canvasgrades' in file_path :
        _, res = dbx.files_download(file_path)
        with io.BytesIO(res.content) as stream:
            df = pd.read_csv(stream)
        pd.set_option('display.max_columns', None)
        year_code = file_path[int(file_path.find('.csv'))-4:int(file_path.find('.csv'))]
        print(' - - - grades file')
        df_term = set_term_code(file_path,year_code)
        df_term_desc = get_term_desc(df_term)
        df['Term Code'] = df_term
        df['Term Desc'] = df_term_desc
        df = df.rename({'Canvas_course_id': 'Canvas CID','final_score':'Final Grade',
                        'unique_name':'Uniqname'}, axis='columns')
        df['Canvas CID'] = df['Canvas CID']%1000000
        df['Canvas CID'] = df['Canvas CID'].astype(str)
        dfp_grades = pd.concat([dfp_grades,df])
        

    # check for student course roster data file name to compile course headcount main dataframe
    elif 'myla_student_data' in file_path :
        _, res = dbx.files_download(file_path)
        with io.BytesIO(res.content) as stream:
            df = pd.read_csv(stream)
        pd.set_option('display.max_columns', None)
        year_code = file_path[int(file_path.find('_withcanvas'))-4:int(file_path.find('_withcanvas'))]
        print(' - - - student course file')
        df_term = set_term_code(file_path,year_code)
        df_term_desc = get_term_desc(df_term)
        df['Term Code'] = df_term
        df['Term Desc'] = df_term_desc
        if df_term == '201920' :
            df = df.rename({'Canvas Course': 'CanvasCourseName'}, axis='columns')
            df['CanvasCourseID_short'] = ' '
        if df_term == '202020' or df_term == '202110' :
            df = df.rename({'CanvasCousreName': 'CanvasCourseName'}, axis='columns')
        df = df[['Campus ID', 'Sex', 'International or Domestic','Acad Level BOT', 'Cum GPA', 
                   'CanvasCourseName', 'CanvasCourseID_short', 'Term Code','Term Desc']]
        df = df.rename({'CanvasCourseName': 'Canvas CName','International or Domestic':'Intl/Dom',
                           'Acad Level BOT':'Class Level','CanvasCourseID_short':'Canvas CID',
                           'Campus ID':'Uniqname'}, axis='columns')
        df['Canvas CID'] = df['Canvas CID'].astype(str)
        df['Uniqname'] = df['Uniqname'].str.lower()
        dfp_main = pd.concat([dfp_main,df])
    
        
    # check for myla log data file name to analyze tool usage data, later linked to student course data
    elif 'myla_event_log' in file_path and 'grade' not in file_path :
        _, res = dbx.files_download(file_path)
        with io.BytesIO(res.content) as stream:
            df = pd.read_csv(stream)
        pd.set_option('display.max_columns', None)
        year_code = file_path[int(file_path.find('.csv'))-4:int(file_path.find('.csv'))]
        print(' - - - event log file')
        df_term = set_term_code(file_path,year_code)
        df_term_desc = get_term_desc(df_term)
        df['Term Code'] = df_term
        df['Term Desc'] = df_term_desc
        df = df.rename({'id':'Click LID','timestamp':'Click Timestamp','action':'Module',
                       'username':'Uniqname','course_id':'Canvas CID','extra':'Extra'}, axis='columns')
        df['Canvas CID'] = df['Canvas CID']%1000000
        df['Canvas CID'] = df['Canvas CID'].astype(str)
        df['Click LID'] = df['Click LID'].astype(str)
        df.drop(columns=['user_id','content_type_id','object_id'],inplace=True)
        dfp_logs = pd.concat([dfp_logs,df])


    else:
        print(' - - - other file')
        print(file_path)
        
        
print('\nBREAK')

 - - - other file
/myla data identified subset/reference only
 - - - grades file
 - - - student course file
 - - - event log file
 - - - grades file
 - - - grades file
 - - - grades file
 - - - event log file
 - - - student course file
 - - - event log file
 - - - event log file
 - - - student course file
 - - - student course file
 - - - course map file
 - - - other file
/myla data identified subset/winter2019_course_map.csv
 - - - student map file

BREAK


In [18]:
# - - - - - - - - - - - - - - - - Set up Main Student/Course file - - - - - - - - - - - - - - - - - - - - - - - -

df_main = dfp_main.dropna()
df_main = df_main.drop_duplicates()
# remove specific course that should not be included in analysis
df_main = df_main[df_main['Canvas CID'] != '322656']
# fill in missing Canvas course ID from course map data set for Winter 2019 term data
df_main = pd.merge(df_main,df_cmap,on='Canvas CName',how='left')
df_main['Canvas CID'] = np.where(
    df_main['Canvas CID_y'].isna(), df_main['Canvas CID_x'], 
     np.where(df_main['Canvas CID_y'].notna(),  df_main['Canvas CID_y'], 'Unknown')
)
df_main.drop(columns=['Canvas CID_y','Canvas CID_x'],inplace=True)
# truncate cumulative GPA column to 2 decimal places for consistency
df_main['Cum GPA'] = ((df_main['Cum GPA']*100).astype(int).astype(float))/100
df_main


,Uniqname,Sex,Intl/Dom,Class Level,Cum GPA,Canvas CName,Term Code,Term Desc,Canvas CID
0,abdsul,Male,International,Grad Mastr,3.67,HMP 654 001 FA 2019,202010,Fall 2019,317173
1,aileenz,Female,Domestic,Grad Mastr,3.73,HMP 654 001 FA 2019,202010,Fall 2019,317173
2,alexfox,Male,Domestic,Grad Mastr,3.87,HMP 654 001 FA 2019,202010,Fall 2019,317173
3,angiemm,Female,Domestic,Grad Mastr,3.87,HMP 654 001 FA 2019,202010,Fall 2019,317173
4,asbrandt,Male,Domestic,Grad Mastr,3.77,HMP 654 001 FA 2019,202010,Fall 2019,317173
...,...,...,...,...,...,...,...,...,...
4582,vessecce,Male,Domestic,Sophomore,2.95,EECS 215 002 WN 2019,201920,Winter 2019,268094
4583,xuanting,Male,Domestic,Junior,3.60,EECS 215 002 WN 2019,201920,Winter 2019,268094
4584,yunzen,Male,International,Sophomore,3.43,EECS 215 002 WN 2019,201920,Winter 2019,268094
4585,ziyangji,Male,International,Junior,3.83,EECS 215 002 WN 2019,201920,Winter 2019,268094


In [19]:
# - - - - - - - - - - - - - - - - Set up Student/Course Grades file - - - - - - - - - - - - - - - - - - - - - - - -

df_grades = dfp_grades.dropna()
df_grades = df_grades.drop_duplicates()
# remove specific course that should not be included in analysis
df_grades = df_grades[df_grades['Canvas CID'] != '322656']
# split uniqnames that have full email address, remove @url.com from string, only keep the name = uniqname
df_grades['Uniqname'] = df_grades['Uniqname'].apply(lambda x:x.split("@")[0])
# calculate equivalent courese GPA from 100 point scale course grade
df_grades['Course GPA'] = ((df_grades['Final Grade'].astype(int)/10-5).astype(int) +
                              (((df_grades['Final Grade'].astype(int)%10+.5)*.3).astype(int)-1)*.33)
df_grades.drop(columns=['current_score'],inplace=True)
df_grades['Course GPA'] = df_grades['Course GPA'].clip(0,4)
# truncate course GPA and Final Grade columns to 2 decimal places for consistency
df_grades['Course GPA'] = ((df_grades['Course GPA']*100).astype(int).astype(float))/100
df_grades['Final Grade'] = ((df_grades['Final Grade']*100).astype(int).astype(float))/100
df_grades


,Canvas CID,Uniqname,Final Grade,Term Code,Term Desc,Course GPA
0,320481,rkulk,98.50,202010,Fall 2019,4.00
1,310629,hoffcar,87.07,202010,Fall 2019,3.33
2,353785,thielj,93.28,202010,Fall 2019,4.00
3,309849,kebyee,97.10,202010,Fall 2019,4.00
4,309849,apokrief,96.13,202010,Fall 2019,4.00
...,...,...,...,...,...,...
1009,343687,sophjac,96.02,202020,Winter 2020,4.00
1010,333620,oshulman,97.56,202020,Winter 2020,4.00
1011,343687,nlampa,75.26,202020,Winter 2020,2.00
1012,347285,psiegel,72.59,202020,Winter 2020,1.67


In [20]:
# - - - - - - - - - - - - - - - - Set up MyLA Log Data file - - - - - - - - - - - - - - - - - - - - - - - -

df_logs = dfp_logs.dropna()
df_logs = df_logs.drop_duplicates()
# remove specific course that should not be included in analysis
df_logs = df_logs[df_logs['Canvas CID'] != '322656']
df_logs = df_logs.sort_values(by=['Term Code','Uniqname','Click Timestamp'])
# rename/recode the module column fields to shorter text
# may consider consolidating some of these, to make 4 total columns
df_logs['Module'] = df_logs['Module'].replace({'VIEW_FILE_ACCESS': 'Rsrc Acs',
                                                'VIEW_GRADE_DISTRIBUTION': 'Grade Dist',
                                                'VIEW_ASSIGNMENT_PLANNING':'Asgmt Plan',
                                                'VIEW_SET_DEFAULT':'Set Def',
                                                'VIEW_RESOURCE_ACCESS':'Rsrc Acs',
                                                'VIEW_ASSIGNMENT_PLANNING_WITH_GOAL_SETTING':'Asgmt Plan'
                                              })
# manually enter start and end dates for the 4 terms in our scope
df_logs['Term Start Date'] = df_logs['Term Code'].replace({
                                                '201920': '2019-01-09',
                                                '202010': '2019-09-03',
                                                '202020': '2020-01-08',
                                                '202110': '2020-08-31'
                                              })
df_logs['Term Start Date'] = df_logs['Term Start Date'].apply(pd.to_datetime).dt.date
df_logs['Term End Date'] = df_logs['Term Code'].replace({
                                                '201920': '2019-05-02',
                                                '202010': '2019-12-20',
                                                '202020': '2020-04-30',
                                                '202110': '2020-12-18'
                                              })
df_logs['Term End Date'] = df_logs['Term End Date'].apply(pd.to_datetime).dt.date
# create separate columns for timestamp split into both 'date' and 'time' for easier manipulation
df_logs['Click Date'] = pd.to_datetime(df_logs['Click Timestamp']).dt.date
df_logs['Click Time'] = pd.to_datetime(df_logs['Click Timestamp']).dt.time
df_logs



,Click LID,Click Timestamp,Module,Extra,Uniqname,Canvas CID,Term Code,Term Desc,Term Start Date,Term End Date,Click Date,Click Time
1031,2526,2019-02-27 17:08:22.354521,Rsrc Acs,"{""week_num_start"":5,""week_num_end"":7,""grade"":""...",aaamanda,274433,201920,Winter 2019,2019-01-09,2019-05-02,2019-02-27,17:08:22.354521
2469,3965,2019-04-02 17:27:35.386178,Grade Dist,"{""course_id"":""17700000000274433""}",aaamanda,274433,201920,Winter 2019,2019-01-09,2019-05-02,2019-04-02,17:27:35.386178
2470,3966,2019-04-02 17:27:44.897603,Asgmt Plan,"{""course_id"":""17700000000274433"",""percent_sele...",aaamanda,274433,201920,Winter 2019,2019-01-09,2019-05-02,2019-04-02,17:27:44.897603
2471,3967,2019-04-02 17:27:55.079994,Rsrc Acs,"{""week_num_start"":10,""week_num_end"":12,""grade""...",aaamanda,274433,201920,Winter 2019,2019-01-09,2019-05-02,2019-04-02,17:27:55.079994
2474,3970,2019-04-02 17:53:17.57415,Rsrc Acs,"{""week_num_start"":10,""week_num_end"":12,""grade""...",aaamanda,274433,201920,Winter 2019,2019-01-09,2019-05-02,2019-04-02,17:53:17.574150
...,...,...,...,...,...,...,...,...,...,...,...,...
3217,21044,2020-12-16 01:57:14.737655,Rsrc Acs,"{""week_num_start"": 1, ""week_num_end"": 16, ""gra...",zwiernik,390340,202110,Fall 2020,2020-08-31,2020-12-18,2020-12-16,01:57:14.737655
3218,21045,2020-12-16 01:57:36.094124,Asgmt Plan,"{""course_id"": 17700000000390340, ""percent_sele...",zwiernik,390340,202110,Fall 2020,2020-08-31,2020-12-18,2020-12-16,01:57:36.094124
756,18081,2020-10-09 13:45:19.326067,Grade Dist,"{""course_id"": 17700000000395418, ""show_number_...",zyichuan,395418,202110,Fall 2020,2020-08-31,2020-12-18,2020-10-09,13:45:19.326067
1445,18929,2020-11-05 01:55:27.890765,Grade Dist,"{""course_id"": 17700000000395418, ""show_number_...",zyichuan,395418,202110,Fall 2020,2020-08-31,2020-12-18,2020-11-05,01:55:27.890765


In [21]:
# - - - - - - - - - - - - - - - - Anonymize Grades File - - - - - - - - - - - - - - - - - - - - - - - -

dfa_grades = pd.merge(df_grades,df_smap,on='Uniqname',how='left')
dfa_grades.drop(columns=['Uniqname'],inplace=True)
dfa_grades = dfa_grades[['Term Code','Term Desc','Canvas CID','Student SID','Final Grade','Course GPA']]
dfa_grades = dfa_grades.sort_values(by=['Term Code','Canvas CID','Student SID'])
dfa_grades = dfa_grades.drop_duplicates()
dfa_grades




,Term Code,Term Desc,Canvas CID,Student SID,Final Grade,Course GPA
2918,201920,Winter 2019,261521,1200,98.29,4.00
2749,201920,Winter 2019,261521,1245,92.87,3.67
3391,201920,Winter 2019,261521,1301,94.52,4.00
2716,201920,Winter 2019,261521,1351,91.36,3.67
2963,201920,Winter 2019,261521,1478,90.65,3.67
...,...,...,...,...,...,...
1938,202110,Fall 2020,428469,4853,90.50,3.67
1514,202110,Fall 2020,428469,4858,98.75,4.00
1775,202110,Fall 2020,428469,4887,84.50,3.00
1508,202110,Fall 2020,428469,4933,85.75,3.00


In [22]:
# - - - - - - - - - - - - - - - - Anonymize Main File - - - - - - - - - - - - - - - - - - - - - - - -

dfa_main = pd.merge(df_main,df_smap,on='Uniqname',how='left')
dfa_main.drop(columns=['Uniqname'],inplace=True)
dfa_main = dfa_main[['Term Code','Term Desc','Canvas CID','Student SID',
                     'Sex','Intl/Dom','Class Level','Cum GPA','Canvas CName']]
dfa_main = dfa_main.sort_values(by=['Term Code','Canvas CID','Student SID'])
dfa_main = dfa_main.drop_duplicates()
dfa_main

,Term Code,Term Desc,Canvas CID,Student SID,Sex,Intl/Dom,Class Level,Cum GPA,Canvas CName
3990,201920,Winter 2019,261521,1200,Female,Domestic,Senior,3.93,MOVESCI 340 001 WN 2019
4011,201920,Winter 2019,261521,1245,Male,Domestic,Senior,3.72,MOVESCI 340 001 WN 2019
4018,201920,Winter 2019,261521,1301,Female,Domestic,Junior,3.57,MOVESCI 340 001 WN 2019
4026,201920,Winter 2019,261521,1351,Female,Domestic,Senior,3.26,MOVESCI 340 001 WN 2019
4038,201920,Winter 2019,261521,1478,Female,Domestic,Senior,3.61,MOVESCI 340 001 WN 2019
...,...,...,...,...,...,...,...,...,...
2525,202110,Fall 2020,428469,4853,Female,Domestic,Senior,3.41,IOE / MFG 461 FA 2020
2540,202110,Fall 2020,428469,4858,Female,Domestic,Grad Mastr,4.00,IOE / MFG 461 FA 2020
2567,202110,Fall 2020,428469,4887,Male,International,Grad Mastr,3.57,IOE / MFG 461 FA 2020
2584,202110,Fall 2020,428469,4933,Male,Domestic,Grad Mastr,3.34,IOE / MFG 461 FA 2020


In [23]:
# - - - - - - - - - - - - - - - - Anonymize MyLA Log File - - - - - - - - - - - - - - - - - - - - - - - -

dfa_logs = pd.merge(df_logs,df_smap,on='Uniqname',how='left')
dfa_logs.drop(columns=['Uniqname'],inplace=True)
dfa_logs = dfa_logs[dfa_logs['Student SID'].notna()]
dfa_logs['Student SID'] = dfa_logs['Student SID'].astype(int).astype(str)
dfa_logs = dfa_logs[['Term Code','Term Desc','Canvas CID','Student SID','Click LID','Click Timestamp',
                     'Module','Extra','Click Date','Click Time','Term Start Date','Term End Date']]
dfa_logs = dfa_logs.sort_values(by=['Term Code','Canvas CID','Student SID','Click Timestamp'])
dfa_logs = dfa_logs.drop_duplicates()
dfa_logs
# dfa_logs.to_clipboard()

,Term Code,Term Desc,Canvas CID,Student SID,Click LID,Click Timestamp,Module,Extra,Click Date,Click Time,Term Start Date,Term End Date
604,201920,Winter 2019,261521,1200,5937,2019-04-27 18:39:57.318287,Grade Dist,"{""course_id"":""17700000000261521""}",2019-04-27,18:39:57.318287,2019-01-09,2019-05-02
605,201920,Winter 2019,261521,1200,6161,2019-05-01 13:38:53.650448,Grade Dist,"{""course_id"":""17700000000261521""}",2019-05-01,13:38:53.650448,2019-01-09,2019-05-02
2584,201920,Winter 2019,261521,1245,1554,2019-02-14 15:03:31.502682,Grade Dist,"{""course_id"":""17700000000261521""}",2019-02-14,15:03:31.502682,2019-01-09,2019-05-02
2870,201920,Winter 2019,261521,1301,2308,2019-02-25 18:35:53.071852,Grade Dist,"{""course_id"":""17700000000261521""}",2019-02-25,18:35:53.071852,2019-01-09,2019-05-02
2871,201920,Winter 2019,261521,1301,3653,2019-03-26 14:13:19.0583,Grade Dist,"{""course_id"":""17700000000261521""}",2019-03-26,14:13:19.058300,2019-01-09,2019-05-02
...,...,...,...,...,...,...,...,...,...,...,...,...
13927,202110,Fall 2020,428469,4997,18657,2020-10-26 21:21:42.399691,Rsrc Acs,"{""week_num_start"": 1, ""week_num_end"": 16, ""gra...",2020-10-26,21:21:42.399691,2020-08-31,2020-12-18
13928,202110,Fall 2020,428469,4997,18658,2020-10-26 21:21:48.086091,Rsrc Acs,"{""week_num_start"": 1, ""week_num_end"": 15, ""gra...",2020-10-26,21:21:48.086091,2020-08-31,2020-12-18
13929,202110,Fall 2020,428469,4997,18659,2020-10-26 21:21:48.719235,Rsrc Acs,"{""week_num_start"": 1, ""week_num_end"": 14, ""gra...",2020-10-26,21:21:48.719235,2020-08-31,2020-12-18
13930,202110,Fall 2020,428469,4997,18660,2020-10-26 21:21:51.0672,Asgmt Plan,"{""course_id"": 17700000000428469, ""percent_sele...",2020-10-26,21:21:51.067200,2020-08-31,2020-12-18


In [24]:
# - - - - - - - - - - - - - - - - Print Anonymized Files to csv - - - - - - - - - - - - - - - - - - - - - - - -

# to print dataframe as csv out to file on desktop, use following code
out_file_path_grades = Path('/Users/sticker/Desktop/myla_outputs/myla_grades_anon.csv')
out_file_path_grades.parent.mkdir(parents=True, exist_ok=True)
out_file_path_students = Path('/Users/sticker/Desktop/myla_outputs/myla_students_anon.csv')
out_file_path_students.parent.mkdir(parents=True, exist_ok=True)
out_file_path_logs = Path('/Users/sticker/Desktop/myla_outputs/myla_logs_anon.csv')
out_file_path_logs.parent.mkdir(parents=True, exist_ok=True)


dfa_logs.to_csv(out_file_path_logs,index=False)
dfa_main.to_csv(out_file_path_students,index=False)
dfa_grades.to_csv(out_file_path_grades,index=False)


In [173]:
# - - - - - - - - - - - - - - - - Unqiname Count comparisons - - - - - - - - - - - - - - - - - - - - - - - -
# - - - - - for validation, not part of in/out joining process - - - - - -




grades_names = df_grades['Uniqname'].unique()
grades_names = set(grades_names)

main_names = df_main['Uniqname'].unique()
main_names = set(main_names)

log_names = df_logs['Uniqname'].unique()
log_names = set(log_names)

combo_names = main_names.union(grades_names)
# print(combo_names.difference(log_names))
print(len(log_names.difference(combo_names)))


# combo_names = pd.DataFrame(combo_names)
# combo_names.to_clipboard()

# log_names = pd.DataFrame(log_names)
# log_names


n_stud = df_main['Uniqname'].nunique()
print('df_main:',n_stud)
n_stud = df_grades['Uniqname'].nunique()
print('df_grades:',n_stud)
n_stud = df_logs['Uniqname'].nunique()
print('df_logs:',n_stud)
n_stud = df_smap['Uniqname'].nunique()
print('df_smap:',n_stud)

# print(len(main_names.difference(grades_names)))
# print(len(grades_names.difference(main_names)))

n_stud_course_grades = df_grades.groupby(['Term Code','Canvas CID']).nunique()['Uniqname']
n_stud_course_main = df_main.groupby(['Term Code','Canvas CID']).nunique()['Uniqname']
n_stud_course_logs = df_logs.groupby(['Term Code']).nunique()['Uniqname']

# main
# grades_names
# combo_names = pd.merge(main_names,grades_names,how='left',indicator=True)
# combo_names.to_clipboard()



37
